In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from lightgbm import LGBMClassifier, LGBMClassifier
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from scipy.stats import uniform, randint

### 결측치 제거 

In [2]:
# 데이터 가져오기
train_data = pd.read_csv("train.csv")
train_data_drop = train_data.dropna()

print(train_data_drop.info())


'''
train_data_drop = train_data.dropna()

train_drop = train_data_drop.iloc[:, 1:27]
#train_drop = train_drop.drop(['country'], axis=1)
target_drop = train_data_drop.iloc[:, 69]

'''
# main_data_drop = pd.concat([idx1to26_drop, idx41to56_drop], axis=1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12639 entries, 0 to 14999
Data columns (total 70 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         12639 non-null  int64  
 1   Q1            12639 non-null  float64
 2   Q2            12639 non-null  float64
 3   Q3            12639 non-null  float64
 4   Q4            12639 non-null  float64
 5   Q5            12639 non-null  float64
 6   Q6            12639 non-null  float64
 7   Q7            12639 non-null  float64
 8   Q8            12639 non-null  float64
 9   Q9            12639 non-null  float64
 10  Q10           12639 non-null  float64
 11  Q11           12639 non-null  float64
 12  Q12           12639 non-null  float64
 13  Q13           12639 non-null  float64
 14  Q14           12639 non-null  float64
 15  Q15           12639 non-null  float64
 16  Q16           12639 non-null  float64
 17  Q17           12639 non-null  float64
 18  Q18           12639 non-nu

"\ntrain_data_drop = train_data.dropna()\n\ntrain_drop = train_data_drop.iloc[:, 1:27]\n#train_drop = train_drop.drop(['country'], axis=1)\ntarget_drop = train_data_drop.iloc[:, 69]\n\n"

In [3]:
data = pd.concat((train_data_drop.iloc[:,1:27], train_data_drop.iloc[:,41:57]), axis=1)
td = train_data_drop['nerdiness'].to_numpy()
td_1 = td.reshape(-1,1)

print(data.shape)
print(td_1.shape)
train_input, test_input, train_target, test_target = train_test_split(data, td_1, test_size=0.2, random_state=42)

(12639, 42)
(12639, 1)


### 랜덤 포레스트 작업

In [4]:
rf = RandomForestClassifier(n_jobs=-1)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

1.0 0.764316346192867


### 특성 중요도 추가 

In [5]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.0446067  0.04558954 0.02472237 0.04154009 0.02934966 0.06287147
 0.02662336 0.03092847 0.02714214 0.02494002 0.03926147 0.03588438
 0.0308507  0.03761949 0.04867032 0.0321387  0.02606441 0.03242774
 0.03337465 0.03281496 0.03172768 0.03150552 0.03085806 0.02591732
 0.03325445 0.03210441 0.00282946 0.0046758  0.01233251 0.00241465
 0.00328402 0.00715209 0.01011922 0.01261909 0.00626663 0.00299551
 0.01173111 0.01109897 0.01019522 0.0052868  0.00278437 0.00142646]


### 하이퍼파라미터 튜닝 / 랜덤서치

In [6]:
rgen = randint(0,10)
rgen.rvs(10)

array([4, 1, 3, 1, 8, 5, 0, 3, 5, 4])

In [7]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 96, 102, 109, 105,  92, 102, 109,  90,  94, 101]))

In [8]:
ugen = uniform(0,1)
ugen.rvs(10)


array([0.43453821, 0.26886436, 0.79266187, 0.67999027, 0.23262216,
       0.51978178, 0.79639487, 0.29713513, 0.36829007, 0.65083476])

In [9]:
params ={'min_impurity_decrease': uniform(0.0001, 0.001), 'max_depth': randint(20,50), 'min_samples_split': randint(2,25), 'min_samples_leaf': randint(1,25),}

In [10]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)

In [11]:
gs.fit(train_input, train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 26, 'min_impurity_decrease': 0.0002743664290049914, 'min_samples_leaf': 1, 'min_samples_split': 9}
0.6979519893137915


In [12]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.7132120253164557


### OOB 점수 출력

In [13]:
# dt = DecisionTreeClassifier(oob_score=True, n_jobs=-1, random_state=42)
# # rf.fit(train_input, train_target)
# print(dt.oob_score_)

### 테스트 데이터 가져오기

In [14]:
test_data = pd.read_csv("test.csv")
test_data_rpm = test_data.fillna(test_data.mean())

In [15]:
# print(test_data_rpm.shape)

test_data_input = pd.concat((test_data_rpm.iloc[:,1:27], test_data_rpm.iloc[:,41:57]), axis=1)
print(test_data_input.shape)

(35452, 42)


### 모델링_Random Search

In [16]:
# rf.fit(train_input, train_target)
dt_pred= dt.predict(test_data_input)

### 제출 파일 생성

In [17]:
submission = pd.read_csv('sample_submission.csv')

In [18]:
submission["nerdiness"] = dt_pred
submission.to_csv("baseline.csv", index = False)